Preprocessing
===



In [1]:
from constants import *  # bad practice but YOLO
import preproc

In [2]:
import constants
reload(constants);
from constants import *

In [3]:
reload(preproc);
y_tr, x_tr = preproc.read_data(TRAIN_FILE, #filename
                              'subreddit', #label field
                              preprocessor=preproc.tokenize_and_downcase) #your preprocessor

y_te, x_te = preproc.read_data(TEST_FILE, #filename
                              'subreddit', #label field
                              preprocessor=preproc.tokenize_and_downcase) #your preprocessor

In [4]:
reload(preproc);
# vocab = preproc.create_vocab(x_tr, x_te)
counts = preproc.get_vocab_counts(x_tr, x_te)
vocab = [w for w, _ in counts.most_common(10000)]
vocab.append(OFFSET)
K = len(vocab)

In [5]:
reload(preproc);
mapping = preproc.create_mapping(vocab, K)

In [6]:
# Now, we'll turn all of our input features into feature vectors
import numpy as np

Y_tr = np.array(y_tr)
X_tr = [preproc.feature_vector(bow, vocab, K, mapping) for bow in x_tr]

Y_te = np.array(y_te)
X_te = [preproc.feature_vector(bow, vocab, K, mapping) for bow in x_te]

Learning
=========

In [7]:
import learner

### First, let's try logistic regression

In [15]:
%%time
reload(learner);
lr = learner.trainLR(X_tr, Y_tr)

CPU times: user 26.7 s, sys: 85.1 ms, total: 26.7 s
Wall time: 26.8 s


In [16]:
print learner.accuracy(lr, X_tr, Y_tr)
print learner.accuracy(lr, X_te, Y_te)

1.0
0.724


### Now, let's try a neural network (also called Multi-Layer Perceptron)

In [26]:
%%time
reload(learner);
mlp = learner.trainMLP(X_tr, Y_tr)

CPU times: user 1min 49s, sys: 159 ms, total: 1min 49s
Wall time: 1min 50s


In [27]:
print learner.accuracy(mlp, X_tr, Y_tr)
print learner.accuracy(mlp, X_te, Y_te)

1.0
0.696


### Now, let's try Naive Bayes

In [17]:
%%time
reload(learner);
nb = learner.trainNB(X_tr, Y_tr)

CPU times: user 481 ms, sys: 83.8 ms, total: 565 ms
Wall time: 574 ms


In [18]:
print learner.accuracy(nb, X_tr, Y_tr)
print learner.accuracy(nb, X_te, Y_te)

0.98575
0.652


# Dimensionality Reduction

We've had some pretty slow training times for LR and NN. Can we do better?


## RP - Random Projections

In [19]:
%%time
reload(learner);
# Random Projections
rp_tr, rp_te = learner.get_RP_features(X_tr, X_te, n_components=4000)

CPU times: user 12.4 s, sys: 116 ms, total: 12.5 s
Wall time: 12.6 s


### Logistic Regression with RP

In [20]:
%%time
lr_RP = learner.trainLR(rp_tr, Y_tr)


CPU times: user 11.4 s, sys: 21.6 ms, total: 11.4 s
Wall time: 11.5 s


In [21]:
print learner.accuracy(lr_RP, rp_tr, Y_tr)
print learner.accuracy(lr_RP, rp_te, Y_te)

1.0
0.678


### NN with RP

In [22]:
%%time
mlp_rp = learner.trainMLP(rp_tr, Y_tr)

CPU times: user 41.2 s, sys: 54.2 ms, total: 41.2 s
Wall time: 41.4 s


In [23]:
print learner.accuracy(mlp_rp, rp_tr, Y_tr)
print learner.accuracy(mlp_rp, rp_te, Y_te)

1.0
0.642


### NB with RP

In [28]:
%%time
nb_rp = learner.trainNB(rp_tr, Y_tr)

CPU times: user 268 ms, sys: 16.5 ms, total: 285 ms
Wall time: 288 ms


In [ ]:
print learner.accuracy(nb_rp, rp_tr, Y_tr)
print learner.accuracy(nb_rp, rp_te, Y_te)

0.43825
0.394


## ICA

In [ ]:
%%time
reload(learner);
ica_tr, ica_te = learner.get_ICA_features(X_tr, X_te, n_components=4000)

### LR with ICA

In [ ]:
%time lr_ica = learner.trainLR(ica_tr, Y_tr)
print learner.accuracy(lr_ica, ica_tr, Y_tr)
print learner.accuracy(lr_ica, ica_te, Y_te)


### NB with ICA

In [ ]:
%time nb_ica = learner.trainNB(ica_tr, Y_tr)
print learner.accuracy(nb_ica, ica_tr, Y_tr)
print learner.accuracy(nb_ica, ica_te, Y_te)

### NN with ICA

In [ ]:
%time mlp_ica = learner.trainMLP(ica_tr, Y_tr)
print learner.accuracy(mlp_ica, ica_tr, Y_tr)
print learner.accuracy(mlp_ica, ica_te, Y_te)

# PCA

In [9]:
%%time
reload(learner);
pca_tr, pca_te = learner.get_PCA_features(X_tr, X_te, n_components=4000)

CPU times: user 3min 40s, sys: 1.44 s, total: 3min 41s
Wall time: 3min 42s


### LR with PCA

In [10]:
%time lr_pca = learner.trainLR(pca_tr, Y_tr)
print learner.accuracy(lr_pca, pca_tr, Y_tr)
print learner.accuracy(lr_pca, pca_te, Y_te)

CPU times: user 9.59 s, sys: 16 ms, total: 9.6 s
Wall time: 9.64 s
1.0
0.726


### NB with PCA

In [11]:
%time nb_pca = learner.trainNB(pca_tr, Y_tr)
print learner.accuracy(nb_pca, pca_tr, Y_tr)
print learner.accuracy(nb_pca, pca_te, Y_te)

CPU times: user 196 ms, sys: 29.3 ms, total: 226 ms
Wall time: 229 ms
0.491
0.274


### NN with PCA

In [ ]:
%time mlp_pca = learner.trainMLP(pca_tr, Y_tr)
print learner.accuracy(mlp_pca, pca_tr, Y_tr)
print learner.accuracy(mlp_pca, pca_te, Y_te)

CPU times: user 7.43 s, sys: 34 ms, total: 7.46 s
Wall time: 7.49 s
1.0
0.65
